In [1]:
import captum
import torch
import thermostat
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from captum.attr import IntegratedGradients, FeatureAblation
import onnxruntime as ort
from tqdm import tqdm

import json

In [6]:
from datasets import load_dataset
num_samples = 1000
dataset = load_dataset('imdb', ignore_verifications=True)


Reusing dataset imdb (/home/tim/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
ds_sorted = sorted([dict(idx=i)|x for i,x in enumerate(dataset["test"])], key=lambda x: len(x["text"]))

In [18]:
ds_sorted[20]

{'idx': 17327,
 'text': 'Don\'t waste your time and money on it. It\'s not quite as bad as "Adrenalin", by the same director but that\'s not saying much.',
 'label': 0}

keys should be {text, input_ids, tokens, attributions, idx)

In [20]:
with open("../data/imdb-1000.json", "w") as fp:
    json.dump(ds_sorted, fp)

<_io.TextIOWrapper name='../data/imdb-1000.json' mode='w' encoding='UTF-8'>


In [24]:
tokenizer = AutoTokenizer.from_pretrained("textattack/distilbert-base-uncased-imdb")

In [26]:
samples = []
for data in tqdm(ds_sorted[:1000]):
    input1 = tokenizer(data["text"])
    input_ids = torch.tensor([input1["input_ids"]]).long()
    attention_mask = torch.tensor([input1["attention_mask"]]).long()

    tokenized = tokenizer(data["text"])
    data["input_ids"] = tokenized["input_ids"]
    data["attention_mask"] = tokenized["attention_mask"]
    data["tokens"] = tokenizer.convert_ids_to_tokens(tokenized["input_ids"])
    
    samples.append(data)

100%|██████████| 1000/1000 [00:00<00:00, 2226.71it/s]


In [27]:
with open("../data/imdb-distilbert-1000.json", "w") as fp:
    json.dump(samples, fp)

<_io.TextIOWrapper name='../data/imdb-distilbert-1000.json' mode='w' encoding='UTF-8'>
